In [1]:
from collections import defaultdict
from pathlib import Path

import pandas as pd

import wandb

api = wandb.Api()

In [2]:
# configuration
ENTITY = "ssl-for-plant-phenotyping"
PROJECT = "wheat-head-detection"
METRIC_PREFIX = "test"
EXCLUDE_RUNS_WITH = ["sweep"]
SAVE_FILEPATH = "results/gwhd_results.csv"
Path.cwd().joinpath("results").mkdir(exist_ok=True)

In [3]:
runs = api.runs(ENTITY + "/" + PROJECT)

runs_data = defaultdict(list)
for run in runs:
    run_name = run.name
    if any(item in run_name for item in EXCLUDE_RUNS_WITH):
        continue

    runs_data["name"].append(run.config["experiment_name"])

    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    for k, v in run.summary._json_dict.items():  # pylint: disable=protected-access
        if k.startswith(METRIC_PREFIX):
            runs_data[k[len(METRIC_PREFIX) + 1 :]].append(v)


runs_df = pd.DataFrame(runs_data)
runs_df = runs_df.groupby("name").agg(["mean", "std"])
runs_df.columns = ["_".join(a) for a in runs_df.columns.values]
runs_df.reset_index(inplace=True)
runs_df

,name,map_small_mean,map_small_std,map_75_mean,map_75_std,map_mean,map_std,map_50_mean,map_50_std,mar_10_mean,...,mar_large_mean,mar_large_std,mar_100_mean,mar_100_std,map_large_mean,map_large_std,mar_small_mean,mar_small_std,mar_medium_mean,mar_medium_std
0,densecl_imagenet,0.05088,0.000676,0.17578,0.003736,0.24712,0.003104,0.57008,0.004891,0.09752,...,0.53250,0.002888,0.31412,0.002696,0.36412,0.003622,0.06672,0.000444,0.35042,0.003337
1,densecl_imagenet_benchmark,0.05462,0.001648,0.17964,0.005296,0.24900,0.003302,0.56726,0.004127,0.09700,...,0.52590,0.004836,0.32196,0.003430,0.35266,0.011105,0.07096,0.001843,0.36022,0.003806
2,densecl_inat2021,0.06080,0.002537,0.18680,0.003471,0.25634,0.002374,0.58278,0.004467,0.09844,...,0.52922,0.004401,0.32206,0.002028,0.36300,0.003191,0.07794,0.002730,0.35854,0.001874
3,densecl_inat_plants,0.05816,0.001893,0.19670,0.001751,0.26312,0.000396,0.58752,0.002288,0.10008,...,0.53920,0.003463,0.32874,0.001036,0.36848,0.005880,0.07754,0.000934,0.36650,0.001046
4,densecl_uwfc1_75ep,0.05304,0.001701,0.16606,0.001262,0.23028,0.001052,0.52868,0.001612,0.09346,...,0.48542,0.006497,0.29522,0.001232,0.31402,0.006848,0.06810,0.001821,0.32940,0.001255
5,densecl_uwfc2_437ep,0.05422,0.002889,0.16682,0.003572,0.23106,0.003428,0.52974,0.006766,0.09384,...,0.48808,0.004862,0.29784,0.003632,0.31918,0.006923,0.07352,0.004542,0.33138,0.003736
6,densecl_uwfc3_1668ep,0.05034,0.001565,0.16736,0.001582,0.22990,0.002535,0.52572,0.004567,0.09462,...,0.48722,0.006482,0.29364,0.002327,0.31970,0.005142,0.06400,0.002690,0.32804,0.002485
7,densecl_uwfc3_828ep,0.05532,0.001834,0.16880,0.004202,0.23264,0.003792,0.53220,0.005825,0.09426,...,0.48076,0.006155,0.29884,0.003603,0.31182,0.009604,0.07200,0.002077,0.33358,0.004034
8,mocov2_imagenet,0.04758,0.002230,0.16972,0.002305,0.24088,0.002406,0.55696,0.005017,0.09578,...,0.52360,0.001728,0.30770,0.001546,0.35446,0.005179,0.06166,0.002211,0.34392,0.001680
9,mocov2_imagenet_benchmark,0.05210,0.000656,0.17920,0.003024,0.25004,0.002540,0.56978,0.004508,0.09726,...,0.53418,0.004212,0.31600,0.001940,0.36350,0.003642,0.06842,0.000719,0.35238,0.002352


In [4]:
runs_df.to_csv(SAVE_FILEPATH, index=False)